#Loading and preprocessing

In [ ]:
#checking device for colab
import torch
torch.cuda.get_device_name(device=None)

'Tesla V100-SXM2-16GB'

In [ ]:
#installing packages
!pip install datasets
from datasets import load_dataset
!pip install evaluate
import evaluate
!pip install seqeval
import numpy as np
#! pip install -U accelerate
#! pip install -U transformers
!pip install -U accelerate transformers
from transformers import TrainingArguments, AutoTokenizer, AutoModelForTokenClassification, DataCollatorForTokenClassification, Trainer, pipeline
from transformers.pipelines.pt_utils import KeyDataset
from huggingface_hub import notebook_login
import accelerate
accelerate.__version__

'0.25.0'

In [ ]:
notebook_login()

In [ ]:
dataset = load_dataset('Babelscape/multinerd')

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
dataset.shape

{'train': (2678400, 3), 'validation': (334800, 3), 'test': (335986, 3)}

In [ ]:
dataset.unique("lang")

{'train': ['de', 'en', 'es', 'fr', 'it', 'nl', 'pl', 'pt', 'ru', 'zh'],
 'validation': ['de', 'en', 'es', 'fr', 'it', 'nl', 'pl', 'pt', 'ru', 'zh'],
 'test': ['de', 'en', 'es', 'fr', 'it', 'nl', 'pl', 'pt', 'ru', 'zh']}

In [ ]:
eng_dataset = dataset.filter(lambda x: x["lang"] == "en")

In [ ]:
eng_dataset.shape

{'train': (262560, 3), 'validation': (32820, 3), 'test': (32908, 3)}

In [ ]:
#replaces all entity labels except for PERSON(PER), ORGANIZATION(ORG), LOCATION(LOC), DISEASES(DIS), and ANIMAL(ANIM)
def remove_ner_tags(example):
  accepted_ners = [0, 1, 2, 3, 4, 5, 6, 7, 8, 13, 14]
  list_of_ners = example["ner_tags"]
  for i in range(len(list_of_ners)):
    if list_of_ners[i] not in accepted_ners:
      list_of_ners[i] = 0
  example["ner_tags"] = list_of_ners
  return example

In [ ]:
train_a = eng_dataset["train"]
train_b = eng_dataset["train"].map(remove_ner_tags)

In [ ]:
#a/b testing
for a, b in zip(train_a[1]["ner_tags"], train_b[1]["ner_tags"]):
  print(a, b)

0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
25 0
26 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
25 0
26 0
0 0
0 0
0 0


#Alinging and mapping labels

In [ ]:
model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
tokenizer.is_fast

True

In [ ]:
#Function borrowed from Huggingface.
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            new_labels.append(-100)
        else:
            label = labels[word_id]
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [ ]:
#Function borrowed from Huggingface.
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))
    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [ ]:
train_a = train_a.map(tokenize_and_align_labels, batched=True, remove_columns=train_a.column_names)
train_b = train_b.map(tokenize_and_align_labels, batched=True, remove_columns=train_b.column_names)
eval = eng_dataset["validation"].map(tokenize_and_align_labels, batched=True)
test = eng_dataset["test"].map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/262560 [00:00<?, ? examples/s]

#Defining, training and testing the models

In [ ]:
label_dict = {
    "O": 0,
    "B-PER": 1,
    "I-PER": 2,
    "B-ORG": 3,
    "I-ORG": 4,
    "B-LOC": 5,
    "I-LOC": 6,
    "B-ANIM": 7,
    "I-ANIM": 8,
    "B-BIO": 9,
    "I-BIO": 10,
    "B-CEL": 11,
    "I-CEL": 12,
    "B-DIS": 13,
    "I-DIS": 14,
    "B-EVE": 15,
    "I-EVE": 16,
    "B-FOOD": 17,
    "I-FOOD": 18,
    "B-INST": 19,
    "I-INST": 20,
    "B-MEDIA": 21,
    "I-MEDIA": 22,
    "B-MYTH": 23,
    "I-MYTH": 24,
    "B-PLANT": 25,
    "I-PLANT": 26,
    "B-TIME": 27,
    "I-TIME": 28,
    "B-VEHI": 29,
    "I-VEHI": 30,
  }
inv_label_dict = {v: k for k, v in label_dict.items()}
print(inv_label_dict)

{0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC', 7: 'B-ANIM', 8: 'I-ANIM', 9: 'B-BIO', 10: 'I-BIO', 11: 'B-CEL', 12: 'I-CEL', 13: 'B-DIS', 14: 'I-DIS', 15: 'B-EVE', 16: 'I-EVE', 17: 'B-FOOD', 18: 'I-FOOD', 19: 'B-INST', 20: 'I-INST', 21: 'B-MEDIA', 22: 'I-MEDIA', 23: 'B-MYTH', 24: 'I-MYTH', 25: 'B-PLANT', 26: 'I-PLANT', 27: 'B-TIME', 28: 'I-TIME', 29: 'B-VEHI', 30: 'I-VEHI'}


In [ ]:
metric = evaluate.load("seqeval")

In [ ]:
#Based on HuggingFace. Defines the metrics for evaluation.
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Ignore special tokens and convert numbers to actual labels
    true_labels = [[inv_label_dict.get(l) for l in label if l !=-100] for label in labels]
    true_predictions = [
        [inv_label_dict.get(p) for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
  model_checkpoint,
  id2label=inv_label_dict,
  label2id=label_dict,
  )

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.config.num_labels

31

In [ ]:
def def_args(output_dir):
  args = TrainingArguments(
      output_dir,
      evaluation_strategy="epoch",
      save_strategy="epoch",
      learning_rate=2e-5,
      num_train_epochs=3,
      weight_decay=0.01,
      push_to_hub=True,
  )
  return args

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer = tokenizer)

In [ ]:
def train_and_push(train_set, output_dir):
  trainer = Trainer(
      model=model,
      args=def_args(output_dir),
      train_dataset=train_set,
      eval_dataset=test,
      data_collator=data_collator,
      compute_metrics=compute_metrics,
      tokenizer=tokenizer,
  )
  trainer.train()
  trainer.push_to_hub(commit_message="Training complete")

In [ ]:
train_and_push(train_a, "bert_finetuned_ner_a")
train_and_push(train_b, "bert_finetuned_ner_b")

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.037100,0.047501,0.932209,0.941587,0.936874,0.983634
2,0.016000,0.066623,0.941355,0.945759,0.943552,0.984096
3,0.005100,0.082735,0.936323,0.954104,0.945130,0.984280


Checkpoint destination directory bert_finetuned_ner_a/checkpoint-32820 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory bert_finetuned_ner_a/checkpoint-65640 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory bert_finetuned_ner_a/checkpoint-98460 already exists and is non-empty.Saving will proceed but saved results may be invalid.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.007300,0.422982,0.956827,0.865081,0.908644,0.969951
2,0.003700,0.455255,0.961804,0.868980,0.913039,0.970726
3,0.000400,0.468360,0.963695,0.869448,0.914149,0.970674


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Checkpoint destination directory bert_finetuned_ner_b/checkpoint-32820 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Checkpoint destination directory bert_finetuned_ner_b/checkpoint-65640 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Pr